In [7]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

!pip install ultralytics


Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 44.2 MB/s eta 0:00:00


In [8]:
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
import imutils
import math
from ultralytics import YOLO
import torch

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [9]:
#Параметры камеры

focal_mm = 50
f_2 = 2* focal_mm

sensor_width_mm = 4.92
sensor_height_mm = 2.77
focal_pixel = (focal_mm / sensor_width_mm)*1280

WFOV = math.degrees(2*np.arctan(sensor_width_mm/f_2))
HFOV = math.degrees(2*math.atan(sensor_height_mm/f_2))
focal_pixel = (focal_mm / sensor_width_mm)*1280

obj_real_width = 24

In [10]:
#Get divice
print(torch.cuda.get_device_name())
torch.cuda.set_device(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Tesla T4


In [40]:
#load YOLO model
model = YOLO('yolov8x.pt')
model.to(device=device)

100%|██████████| 131M/131M [00:00<00:00, 242MB/s]


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 80, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(80, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(160, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(400, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_r

In [39]:
#Open video file
cap = cv2.VideoCapture('Video Example.mp4')
writer = None
try:
	prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
				else cv2.CAP_PROP_FRAME_COUNT
	total = int(cap.get(prop))
	print("[INFO] {} total frames in video".format(total))
except:
	print("[INFO] could not determine # of frames in video")

[INFO] 450 total frames in video


In [41]:

#Random point on screen
x_rand_point, y_rand_point = (np.random.randint(400,600),(np.random.randint(300,400)))

count = 0
while True:
    ret, frame = cap.read()
    if not ret:
       break


    #load frame in model
    detection =  model(frame)[0]
    for data in detection.boxes.data.tolist():
      confidence = data[4] #Вероятность принадлежности к классу
      if float(confidence) < 0.3:
              continue

      #DrawBox
      (xmin, ymin, xmax, ymax) = int(data[0]), int(data[1]), int(data[2]), int(data[3])
      cv2.rectangle(frame, (xmin, ymin) , (xmax, ymax), 255, 2)


      #Определяем класс объекта
      class_id = data[-1]
      label = "{}".format(detection.names.get(class_id))
      cv2.putText(frame, label, (xmin, ymin-15),cv2.FONT_HERSHEY_SIMPLEX, 0.5, 150, 1)

      #Отклонение от точки до центра объекта
      x,y =  (xmin+xmax)//2,(ymin+ymax)//2 #Centre object
      x_dev = abs(x_rand_point - x)
      y_dev = abs(y_rand_point - y)
      cv2.putText(frame,f'X_dev: {x_dev}' , (900,50),cv2.FONT_HERSHEY_COMPLEX, 0.5, 120, 1)
      cv2.putText(frame,f'Y_dev: {y_dev}' , (900,80),cv2.FONT_HERSHEY_COMPLEX, 0.5, 120, 1)

      #Draw circle random point
      cv2.circle(frame, (x_rand_point,y_rand_point),5,40,-1)
      cv2.circle(frame, (x_rand_point,y_rand_point),80,40,1)
      cv2.circle(frame, (x_rand_point,y_rand_point),160,40,1)
      cv2.circle(frame, (x_rand_point,y_rand_point),240,40,1)


      #Object width in px
      obj_w_px = xmax - xmin
      cv2.putText(frame,f'Object_width: {obj_w_px}' , (10,50),cv2.FONT_HERSHEY_COMPLEX, 0.5, 120, 1)

      #Расчет расстояния по ширине объекта
      width_sensor = (sensor_width_mm /1280)*(obj_w_px) #Размер объекта на матрице
      w_dist = round(obj_real_width * focal_mm/width_sensor,2)
      cv2.putText(frame,f'Distanse: {w_dist}' , (10,70),cv2.FONT_HERSHEY_COMPLEX, 0.5, 255, 1)

      #Расчет по угловым размерам
      vector_1= np.array([xmin,ymin,focal_pixel])
      vector_2 = np.array([xmax,ymax,focal_pixel])
      obg_angle = math.degrees(math.acos(vector_1.dot(vector_2)/(np.linalg.norm(vector_1)*(np.linalg.norm(vector_2)))))
      dist = round(obj_real_width/(2*math.tan(math.radians(obg_angle/2))),2)
      cv2.putText(frame,f'Distanse obg_angle : {dist}' , (10,100),cv2.FONT_HERSHEY_COMPLEX, 0.5, 255, 1)


      cv2.line(frame, (x_rand_point,0), (x_rand_point,frame.shape[0]),150,1,1)
      cv2.line(frame, (0,y_rand_point), (frame.shape[1],y_rand_point),150,1,1)

    #cv2_imshow(frame)
    if writer is None:
      fourcc = cv2.VideoWriter_fourcc(*"XVID")
      writer = cv2.VideoWriter( 'Video.mp4', fourcc, 25,(frame.shape[1], frame.shape[0]))
    print(count)
    count+=1
    writer.write(frame)

print("[INFO] cleaning up...")
writer.release()
cap.release()


0: 384x640 1 airplane, 74.6ms
Speed: 2.2ms preprocess, 74.6ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)
0

0: 384x640 1 airplane, 61.7ms
Speed: 3.9ms preprocess, 61.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
1

0: 384x640 1 airplane, 37.8ms
Speed: 1.8ms preprocess, 37.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
2

0: 384x640 1 airplane, 37.7ms
Speed: 1.8ms preprocess, 37.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
3

0: 384x640 1 airplane, 37.4ms
Speed: 1.8ms preprocess, 37.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
4

0: 384x640 1 airplane, 36.5ms
Speed: 1.8ms preprocess, 36.5ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)
5

0: 384x640 1 airplane, 35.6ms
Speed: 1.7ms preprocess, 35.6ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
6

0: 384x640 1 airplane, 33.0ms
Speed: 2.2ms preprocess, 33.0ms inference, 1.7ms postproces